# Improved Bonferroni 

Some simple functions to manually find the individual confidence intervals needed to obtain a given family wise error rate in a posthoc analysis with treatments groups all the same size.  I consider Normal and Cauchy case.  See corresponding post at link below.

Jonathan Landy

http://efavdb.com/bonferroni-correction-for-multiple-pairwise-comparison-tests/

## Load packages

In [1]:
from scipy.special import erf
from scipy.integrate import quad
from scipy.misc import comb
import numpy as np
import math

## Normal statistics

In [44]:
# Step 1 -- find the k value that corresponds to the Bonferroni approximation
# Notice that the resulting p_fam is too large; Bonferroni is conservative.
N = 3
k = 2.394
p_fam_target = 0.95

# Define the individual pair and family wise integrands.
f_ind = lambda x: np.exp(-x**2 / 4) / np.sqrt(4 * math.pi)
f_fam = lambda x: ( N * np.exp(-x**2 / 2) / np.sqrt(2 * math.pi)
               *np.exp((N-1) * 
                        np.log( (erf(k + x / np.sqrt(2.0)) - erf(x / np.sqrt(2.0))) / 2.0) )) 



print 'Bonferroni target p for individual tests is %2.4f' %(1 -  (1 - p_fam_target) / comb(N,2))
print 'With k set to present value, individual test p values are %2.4f' %(quad(f_ind, -k*np.sqrt(2), k* np.sqrt(2))[0])
result = quad(f_fam, -np.inf, np.inf)
print 'p_fam actual: %2.4f'%(result[0])

Bonferroni target p for individual tests is 0.9833
With k set to present value, individual test p values are 0.9833
p_fam actual: 0.9560


/Users/jonathanlandy/my-venv/lib/python2.7/site-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in log
  # This is added back by InteractiveShellApp.init_path()


In [52]:
# Step 2 -- find the k value that gives the desired family-wise error rate
#     Notice that the individual pair tests are less stringent than Bonferroni.

k = 2.344
 

result = quad(f_fam, -np.inf, np.inf)
print 'p_individual = %2.4f' %(quad(f_ind, -k * np.sqrt(2), k * np.sqrt(2))[0])
print 'p_fam actual: %2.4f'%(result[0])


p_individual = 0.9809
p_fam actual: 0.9500


/Users/jonathanlandy/my-venv/lib/python2.7/site-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in log
  # This is added back by InteractiveShellApp.init_path()


In [55]:
# Check accuracy of the normal case

mu, sigma = 0, 1.0 # mean and standard deviation
count = 0
tot = 10**7
for i in range(tot):
    s = np.random.normal(mu, sigma, N)
    if max(s) - min(s) > k * np.sqrt(2):
        count +=1
        
count / float(tot)

0.049948

## Cauchy statistics

In [258]:
# Cauchy exact

N = 16
k = 110

# Exact probability of no type 1 error
f = lambda x: ((N / math.pi) / (1 + x **2) 
               *np.exp((N-1) * 
                        np.log( (math.atan(k + x) - math.atan(x)) / math.pi) ))
 
result = quad(f, -np.inf, np.inf)
print result
print '%2.4f'%(result[0])


(0.8999212963100884, 4.20742359111235e-09)
0.8999


In [259]:
# Cauchy Bonferroni

N = 16
k = 100

p_f = 0.90
f_ind = lambda x: (2 / math.pi) * (1.0 / (2 ** 2 + x ** 2))

print 'p_desired for individual tests is %2.4f' %(1 -  (1 - p_f) / comb(N,2))
print 'current individual p is %2.4f' %(quad(f_ind, -k, k)[0])

result = quad(f, -np.inf, np.inf)
print 'p_f Bon: %2.4f'%(result[0])


p_desired for individual tests is 0.9992
current individual p is 0.9873
p_f Bon: 0.8896


In [271]:
# Check results for Cauchy

k = 27
N = 4
count = 0
tot = 10**6
for i in range(tot):
    s = np.random.standard_cauchy(N)
    if max(s) - min(s) > k:
        count +=1
        
count / float(tot)


0.100889